In [6]:
%classpath add mvn com.salesforce.transmogrifai transmogrifai-core_2.11 0.6.0

In [13]:
%classpath add mvn org.apache.spark spark-mllib_2.11 2.4.3

In [14]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.sql.functions.udf

import com.salesforce.op._
import com.salesforce.op.features._
import com.salesforce.op.features.types._
import com.salesforce.op.evaluators.Evaluators

import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.sql.functions.udf
import com.salesforce.op._
import com.salesforce.op.features._
import com.salesforce.op.features.types._
import com.salesforce.op.evaluators.Evaluators


In [33]:
import com.salesforce.op.stages.impl.tuning.{DataCutter, DataSplitter}
val randomSeed = 42L
val splitter = DataSplitter(seed = randomSeed)

DataSplitter_00000000002f

In [16]:
import com.salesforce.op.OpWorkflow
import com.salesforce.op.evaluators.Evaluators
import com.salesforce.op.readers.DataReaders

import com.salesforce.op.OpWorkflow
import com.salesforce.op.evaluators.Evaluators
import com.salesforce.op.readers.DataReaders


In [17]:
val conf = new SparkConf().setMaster("local[*]").setAppName("PimaIndiansClassification")
implicit val spark = SparkSession.builder.config(conf).getOrCreate()

org.apache.spark.sql.SparkSession@4a42f1c

In [18]:
case class PimaIndians
(
  numberOfTimesPreg: Double,
  plasmaGlucose: Double,
  bp: Double,
  spinThickness: Double,
  serumInsulin: Double,
  bmi: Double,
  diabetesPredigree : Double,
  ageInYrs : Double,
  piClass: String
)

defined class PimaIndians


In [19]:
val numberOfTimesPreg = FeatureBuilder.Real[PimaIndians].extract(_.numberOfTimesPreg.toReal).asPredictor
val plasmaGlucose = FeatureBuilder.Real[PimaIndians].extract(_.plasmaGlucose.toReal).asPredictor
val bp = FeatureBuilder.Real[PimaIndians].extract(_.bp.toReal).asPredictor
val spinThickness = FeatureBuilder.Real[PimaIndians].extract(_.spinThickness.toReal).asPredictor
val serumInsulin = FeatureBuilder.Real[PimaIndians].extract(_.serumInsulin.toReal).asPredictor
val bmi = FeatureBuilder.Real[PimaIndians].extract(_.bmi.toReal).asPredictor
val diabetesPredigree = FeatureBuilder.Real[PimaIndians].extract(_.diabetesPredigree.toReal).asPredictor
val ageInYrs = FeatureBuilder.Real[PimaIndians].extract(_.diabetesPredigree.toReal).asPredictor
val piClass = FeatureBuilder.Text[PimaIndians].extract(_.piClass.toText).asResponse

Feature(name = piClass, uid = Text_00000000000d, isResponse = true, originStage = FeatureGeneratorStage_00000000000d, parents = [], distributions = [])

In [20]:
 val trainFilePath = "../src/main/resources/PimaIndiansDataset/primaindiansdiabetes.data"

../src/main/resources/PimaIndiansDataset/primaindiansdiabetes.data

In [21]:
import com.salesforce.op.features.FeatureBuilder
import com.salesforce.op.features.types._

import com.salesforce.op.features.FeatureBuilder
import com.salesforce.op.features.types._


In [22]:
import spark.implicits._ 
val trainDataReader = DataReaders.Simple.csvCase[PimaIndians](
      path = Option(trainFilePath)
    )

org.apache.spark.sql.SparkSession$implicits$@2aa956f

In [23]:
import com.salesforce.op.stages.impl.tuning.{DataCutter, DataSplitter}

val features = Seq( numberOfTimesPreg, plasmaGlucose,bp,spinThickness,serumInsulin,
    bmi,diabetesPredigree,ageInYrs).transmogrify()
val randomSeed = 42L
//val splitter = DataSplitter(seed = randomSeed)
DataCutter.getClass
val splitter = DataSplitter(seed = randomSeed)

DataSplitter_000000000010

In [24]:
import org.apache.spark.sql.Encoders

implicit val piEncoder = Encoders.product[PimaIndians]

val piReader = DataReaders.Simple.csvCase[PimaIndians]()
val labels = piClass.indexed()

Feature(name = piClass_1-stagesApplied_RealNN_000000000011, uid = RealNN_000000000011, isResponse = true, originStage = OpStringIndexerNoFilter_000000000011, parents = [Text_00000000000d], distributions = [])

In [25]:
import com.salesforce.op.stages.impl.classification.MultiClassificationModelSelector
import com.salesforce.op.stages.impl.tuning.DataCutter

val cutter = DataCutter(reserveTestFraction = 0.2, seed = randomSeed)

val prediction = MultiClassificationModelSelector
    .withCrossValidation(splitter = Option(cutter), seed = randomSeed)
    .setInput(labels, features).getOutput()

Feature(name = ageInYrs-bmi-bp-diabetesPredigree-numberOfTimesPreg-piClass-plasmaGlucose-serumInsulin-spinThickness_4-stagesApplied_Prediction_00000000001a, uid = Prediction_00000000001a, isResponse = true, originStage = ModelSelector_00000000001a, parents = [RealNN_000000000011,OPVector_00000000000f], distributions = [])

In [26]:
val evaluator = Evaluators.MultiClassification.f1().setLabelCol(labels).setPredictionCol(prediction)

OpMultiClassificationEvaluator_00000000001b

In [27]:
val workflow = new OpWorkflow().setResultFeatures(prediction, labels).setReader(trainDataReader)

com.salesforce.op.OpWorkflow@73d25274

In [28]:
//val workflow = new OpWorkflow().setResultFeatures(prediction1, salePrice).setReader(trainDataReader)
val workflowModel = workflow.train()

com.salesforce.op.OpWorkflowModel@779dac22

In [29]:
val (scores, metrics)  = workflowModel.scoreAndEvaluate(evaluator)

{
  "Precision" : 0.7676517722473605,
  "Recall" : 0.7682291666666667,
  "F1" : 0.7679403609250608,
  "Error" : 0.23177083333333337,
  "ThresholdMetrics" : {
    "topNs" : [ 1, 3 ],
    "thresholds" : [ 0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0 ],
    "correctCounts" : {
      "1" : [ 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 590, 59

In [30]:
val modelInsights = workflowModel.modelInsights(prediction)
val labelSummary = modelInsights.label
println("labelName: " + labelSummary.labelName)
println("rawFeatureName: "+ labelSummary.rawFeatureName)
println("stagesApplied: " + labelSummary.stagesApplied)

labelName: Some(piClass_1-stagesApplied_RealNN_000000000011)
rawFeatureName: List(piClass)
stagesApplied: ArrayBuffer(str2idx_OpStringIndexerNoFilter_000000000011)


null

In [31]:
val modelFeatures = modelInsights.features.flatMap( feature => feature.derivedFeatures)

modelFeatures.foreach(x => println(x.derivedFeatureName + ", " + x.contribution(0))  )
//println("FeatureName, Contribution") 

diabetesPredigree_12, -0.043836894589097855
diabetesPredigree_diabetesPredigree_NullIndicatorValue_13, 0.0
bmi_10, -0.024468078842777487
bmi_bmi_NullIndicatorValue_11, 0.0
plasmaGlucose_2, -0.007777490072994181
plasmaGlucose_plasmaGlucose_NullIndicatorValue_3, 0.0
bp_4, 0.005743323370099911
bp_bp_NullIndicatorValue_5, 0.0
serumInsulin_8, 0.0
serumInsulin_serumInsulin_NullIndicatorValue_9, 0.0
spinThickness_6, 0.0
spinThickness_spinThickness_NullIndicatorValue_7, 0.0
ageInYrs_14, -0.043836894589097855
ageInYrs_ageInYrs_NullIndicatorValue_15, 0.0
numberOfTimesPreg_0, -0.017809435632573617
numberOfTimesPreg_numberOfTimesPreg_NullIndicatorValue_1, 0.0


null

In [32]:
val featureContributions = modelFeatures.map( feature => (feature.derivedFeatureName,
  feature.contribution.map( contribution => math.abs(contribution))
  .foldLeft(0.0) { (max, contribution) => math.max(max, contribution)}))

val sortedContributions = featureContributions.sortBy( contribution => -contribution._2)
sortedContributions.foreach(x => println(x._1 + "," + x._2) )

diabetesPredigree_12,0.16054799085509808
ageInYrs_14,0.16054799085509808
numberOfTimesPreg_0,0.046963977191149406
bmi_10,0.024468078842777487
plasmaGlucose_2,0.012850277986778265
bp_4,0.010888901394568495
diabetesPredigree_diabetesPredigree_NullIndicatorValue_13,0.0
bmi_bmi_NullIndicatorValue_11,0.0
plasmaGlucose_plasmaGlucose_NullIndicatorValue_3,0.0
bp_bp_NullIndicatorValue_5,0.0
serumInsulin_8,0.0
serumInsulin_serumInsulin_NullIndicatorValue_9,0.0
spinThickness_6,0.0
spinThickness_spinThickness_NullIndicatorValue_7,0.0
ageInYrs_ageInYrs_NullIndicatorValue_15,0.0
numberOfTimesPreg_numberOfTimesPreg_NullIndicatorValue_1,0.0


null